# Practice Notebook: Files + PostgreSQL (Codespaces)

This notebook is intentionally **repo-name independent**.

**Important:**  
- Your **GitHub repo name** (e.g., `Data_Science-notebooks`) is *not* your PostgreSQL schema.  
- Your PostgreSQL **database/schema** for this course stays `hi5304` unless your instructor explicitly changes it.

We will:
1. Load a CSV from the repo `data/` folder  
2. Connect to PostgreSQL using a single, reusable connection  
3. Run SQL queries and analyze the results with pandas


## 0) Setup (run once)

This cell:
- finds the repo root (where the `data/` folder lives)
- sets `DATA_DIR` for file loading
- creates a PostgreSQL `engine` for `pd.read_sql(...)`


In [3]:
from pathlib import Path
import pandas as pd

# Project root -> data/
DATA_DIR = Path.cwd().parents[1] / "data"

print("Working directory:", Path.cwd())
print("Data directory:", DATA_DIR)
print("Data directory exists:", DATA_DIR.exists())


Working directory: /workspaces/Data_Science_Coding_Laboratory/lessons/Jupyter
Data directory: /workspaces/Data_Science_Coding_Laboratory/data
Data directory exists: True


In [6]:
from sqlalchemy import create_engine, text
import os

# --- Docker Compose Postgres settings (matches your docker-compose.yml) ---
DB_HOST = os.getenv("DB_HOST", "db")  # service name in docker-compose
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME", "data_science")
DB_USER = os.getenv("DB_USER", "dataScience_user")
DB_PASSWORD = os.getenv("DB_PASSWORD", "data_science")

# SQLAlchemy engine (psycopg3 driver)
engine = create_engine(
    f"postgresql+psycopg://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}",
    pool_pre_ping=True,
)

# Default schema (Postgres default unless you created another)
SCHEMA = os.getenv("DB_SCHEMA", "public")

# Quick connection test (optional but recommended)
with engine.connect() as conn:
    conn.execute(text("SELECT 1;"))

print("✅ Connected to Postgres")
print("Host:", DB_HOST, "DB:", DB_NAME, "Schema:", SCHEMA)


✅ Connected to Postgres
Host: db DB: data_science Schema: public


In [ ]:
%pip install psycopg[binary] sqlalchemy pandas


In [1]:
import psycopg
import sqlalchemy
import pandas as pd

print("All imports worked")


All imports worked


In [17]:
import pandas as pd

def run_sql(query):
    """
    Execute a SQL query and return results as a Pandas DataFrame.
    """
    return pd.read_sql(query, engine)


In [4]:
# Load the patients CSV from the repo's data/ folder
patients = pd.read_csv(DATA_DIR / "patients.csv")

patients.head()


,patient_id,first_name,last_name,date_of_birth,sex,race,ethnicity,zip_code
0,1001,John,Doe,4/12/1968,M,White,Non-Hispanic,75201
1,1002,Maria,Lopez,9/30/1975,F,Hispanic,Hispanic,75204
2,1003,James,Smith,1/18/1982,M,Black,Non-Hispanic,75080
3,1004,Linda,Chen,6/5/1990,F,Asian,Non-Hispanic,75024
4,1005,Robert,Johnson,11/22/1959,M,White,Non-Hispanic,75230


In [7]:
schema = SCHEMA
query = f"""
SELECT *
FROM {schema}.patients
LIMIT 5;
"""

df_patients = pd.read_sql(query, engine)
df_patients


,patient_id,first_name,last_name,date_of_birth,sex,race,ethnicity,zip_code
0,1001,John,Doe,1968-04-12,M,White,Non-Hispanic,75201
1,1002,Maria,Lopez,1975-09-30,F,Hispanic,Hispanic,75204
2,1003,James,Smith,1982-01-18,M,Black,Non-Hispanic,75080
3,1004,Linda,Chen,1990-06-05,F,Asian,Non-Hispanic,75024
4,1005,Robert,Johnson,1959-11-22,M,White,Non-Hispanic,75230


In [8]:
# Shape of the dataset
patients.shape


(5, 8)

In [9]:
# Column names
patients.columns


Index(['patient_id', 'first_name', 'last_name', 'date_of_birth', 'sex', 'race',
       'ethnicity', 'zip_code'],
      dtype='str')

In [10]:
# Basic info
patients.info()


<class 'pandas.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   patient_id     5 non-null      int64
 1   first_name     5 non-null      str  
 2   last_name      5 non-null      str  
 3   date_of_birth  5 non-null      str  
 4   sex            5 non-null      str  
 5   race           5 non-null      str  
 6   ethnicity      5 non-null      str  
 7   zip_code       5 non-null      int64
dtypes: int64(2), str(6)
memory usage: 452.0 bytes


In [12]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path.cwd().parents[1] / "data"

print("Working directory:", Path.cwd())
print("Data directory:", DATA_DIR)
print("Exists:", DATA_DIR.exists())

patients = pd.read_csv(DATA_DIR / "patients.csv")
patients.head()


Working directory: /workspaces/Data_Science_Coding_Laboratory/lessons/Jupyter
Data directory: /workspaces/Data_Science_Coding_Laboratory/data
Exists: True


,patient_id,first_name,last_name,date_of_birth,sex,race,ethnicity,zip_code
0,1001,John,Doe,4/12/1968,M,White,Non-Hispanic,75201
1,1002,Maria,Lopez,9/30/1975,F,Hispanic,Hispanic,75204
2,1003,James,Smith,1/18/1982,M,Black,Non-Hispanic,75080
3,1004,Linda,Chen,6/5/1990,F,Asian,Non-Hispanic,75024
4,1005,Robert,Johnson,11/22/1959,M,White,Non-Hispanic,75230


In [15]:
schema = SCHEMA

query = f"""
SELECT
    p.patient_id,
    p.first_name,
    p.last_name,
    b.reading_date,
    b.systolic,
    b.diastolic,
    b.heart_rate
FROM {schema}.patients p
JOIN {schema}.bp_readings b
  ON p.patient_id = b.patient_id
ORDER BY b.reading_date;
"""

df_bp = pd.read_sql(query, engine)
df_bp.head()



,patient_id,first_name,last_name,reading_date,systolic,diastolic,heart_rate
0,1001,John,Doe,2024-09-01,148,92,78
1,1005,Robert,Johnson,2024-09-05,140,90,75
2,1003,James,Smith,2024-09-10,150,96,82
3,1002,Maria,Lopez,2024-09-15,132,82,70
4,1004,Linda,Chen,2024-09-20,118,76,66


In [18]:
schema = SCHEMA
query = f"""
SELECT
    patient_id,
    COUNT(*) AS medication_count
FROM {schema}.medications
GROUP BY patient_id
ORDER BY medication_count DESC;
"""

df_meds = run_sql(query)
df_meds


,patient_id,medication_count
0,1003,2
1,1002,1
2,1001,1
3,1005,1


In [19]:
schema = SCHEMA
query = f"""
SELECT
    AVG(systolic) AS avg_systolic,
    AVG(diastolic) AS avg_diastolic,
    COUNT(*) AS total_readings
FROM {schema}.bp_readings;
"""

df_summary = run_sql(query)
df_summary


,avg_systolic,avg_diastolic,total_readings
0,137.777778,86.888889,9


In [20]:
df_bp.describe()


,patient_id,systolic,diastolic,heart_rate
count,9.000000,9.000000,9.000000,9.000000
mean,1002.444444,137.777778,86.888889,74.333333
std,1.424001,10.413666,6.790516,5.431390
min,1001.000000,118.000000,76.000000,66.000000
25%,1001.000000,132.000000,82.000000,70.000000
50%,1002.000000,140.000000,88.000000,75.000000
75%,1003.000000,146.000000,92.000000,78.000000
max,1005.000000,150.000000,96.000000,82.000000


In [21]:
df_bp.groupby("patient_id")[["systolic", "diastolic"]].mean()


,systolic,diastolic
patient_id,,
1001,142.0,88.0
1002,130.0,81.0
1003,148.0,95.0
1004,118.0,76.0
1005,140.0,90.0
